In [648]:
import random

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import product
import Cython
from numba import njit

In [649]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [650]:
"""
rho = 0.85
h = 0.016
m = 48
T = 1
M = 6 # cube root of number of unit cells in the L^3 volume, gives N=864 particles
L = (4*6**3)/rho
sigma = 3.405 #Angstrom
epsilon = 119.8 #K
# Todo: Scale time
"""

'\nrho = 0.85\nh = 0.016\nm = 48\nT = 1\nM = 6 # cube root of number of unit cells in the L^3 volume, gives N=864 particles\nL = (4*6**3)/rho\nsigma = 3.405 #Angstrom\nepsilon = 119.8 #K\n# Todo: Scale time\n'

In [651]:
%%cython
import numpy as np
from itertools import product
from tqdm import tqdm

cdef double rho = 0.85
cdef double h = 0.016
cdef int m = 48 # Be aware that m is also defined later outside of the cython
cdef double T = 2.0
cdef int M = 4   # cube root of number of unit cells in the L^3 volume, gives N=864 particles
cdef double L = np.cbrt((4*M**3)/rho)
cdef int num_steps = 1000
cdef double sigma = 3.405 #Angstrom
cdef double epsilon = 119.8 #K

list_pbc = list(product([0, 1, -1], repeat=3))
list_pbc = L*np.array(list_pbc)

def build_fcc():
    """
    :param M: integer. cube root of Number of unit cells
    :return: fcc grid for the L*L cube (np.array of shape (n_points, 3))
    """
    range_values_sc = np.linspace(-L/2, L/2, M+1)
    length_edge = np.abs(range_values_sc[0]-range_values_sc[1]) #L/(rho-1)
    sc_grid = np.array(list(product(range_values_sc, repeat=3)))
    sc_grid = sc_grid[sc_grid[:, 0] != -L/2]
    sc_grid = sc_grid[sc_grid[:, 1] != -L/2]
    sc_grid = sc_grid[sc_grid[:, 2] != -L/2]
    missing_points = []
    for point in sc_grid:
        p1 = [point[0]+0.5*length_edge, point[1], point[2]+0.5*length_edge]
        p2 = [point[0]+0.5*length_edge, point[1]+0.5*length_edge, point[2]]
        p3 = [point[0], point[1]+0.5*length_edge, point[2]+0.5*length_edge]
        missing_points.append(p1)
        missing_points.append(p2)
        missing_points.append(p3)
    missing_points = np.array(missing_points)
    x = sc_grid[:,0]
    y = sc_grid[:,1]
    z = sc_grid[:,2]
    x_m = missing_points[:,0]
    y_m = missing_points[:,1]
    z_m = missing_points[:,2]
    fcc_grid = np.concatenate([sc_grid, missing_points])

    return fcc_grid

r_0 = build_fcc()
N_0 = r_0.shape[0]
v_0 = np.random.normal(0,np.sqrt(T/epsilon*m), (N_0, 3))

list_r = [r_0]
list_v = [v_0]

def f_optimized(double[:] r_i, double[:] r_j):
    cdef double[:] vec_L_min
    cdef double[:] r_ij_candidates = np.zeros(27)

    for i, vec_L in enumerate(list_pbc):
        for d in range(3):
            r_ij_candidates[i] += (r_i[d] - r_j[d] + L*vec_L[d])**2

    cdef int index_L_min = np.argmin(r_ij_candidates)
    vec_L_min = L * list_pbc[index_L_min]
    cdef double r_ij = (r_ij_candidates[index_L_min])

    cdef double f_x = m * (r_i[0] - r_j[0] + vec_L_min[0]) * (r_ij**(-14) + 0.5 * r_ij**(-8))
    cdef double f_y = m * (r_i[1] - r_j[1] + vec_L_min[1]) * (r_ij**(-14) + 0.5 * r_ij**(-8))
    cdef double f_z = m * (r_i[2] - r_j[2] + vec_L_min[2]) * (r_ij**(-14) + 0.5 * r_ij**(-8))
    return np.array([f_x, f_y, f_z])

def arange_without(int i):
    cdef int N = 4 * M**3
    cdef set indices = set(range(N))
    indices.remove(i)
    return np.array(list(indices))

def verlet_step(double[:, :] r, double[:, :] v):
    cdef int i, d
    cdef int j, k
    cdef double[:,:] v_tilde = np.zeros((r.shape[0], 3), order='C')
    cdef double[:,:] r_next = np.zeros((r.shape[0], 3), order='C')
    cdef double[:,:] v_next = np.zeros((r.shape[0], 3), order='C')
    cdef double[:,:] F_r = np.zeros((r.shape[0], 3), order='C')
    cdef double[:,:] f_next = np.zeros((r.shape[0], 3), order='C')
    print("F_r")
    for i in range(r.shape[0]):
        for j in range(3):
            for k in arange_without(i):
                F_r[i,j] += f_optimized(r[i], r[k])[j]
            v_tilde[i,j] += v[i,j] + (h/(2.0*m))*F_r[i,j]
            r_next[i,j] += r[i,j] + h * v_tilde[i,j]
    print("F_next")
    for i in range(r.shape[0]):
        for d in range(3):
            for k in arange_without(i):
                f_next[i,d] += f_optimized(r_next[i], r_next[k])[d]
            v_next[i,d] += v_tilde[i,j] + (h/(2.0*m))*f_next[i,j]
    print("done")
    return r_next, v_next

#def verlet_algorithm(double[:, :] r_0, double[:, :] v_0, int num_steps):
#    r_0 = np.array(r_0)
#    v_0 = np.array(v_0)
#    cdef int t
#    cdef double[:, ::1] list_r = np.zeros((num_steps + 1, r_0.shape[0], 3), order='C')
#    cdef double[:, ::1] list_v = np.zeros((num_steps + 1, v_0.shape[0], 3), order='C')
#    list_r = np.array(list_r)
#    list_v = np.array(list_v)
#    print(list_r.shape)
#    list_r[0] = r_0
#    list_v[0] = v_0
#    print("3")
#    for t in tqdm(range(num_steps)):
#        r_new, v_new = verlet_step(list_r[t], list_v[t])

#        print("Shapes: r_new={}, v_new={}, list_r[t+1]={}, list_v[t+1]={}".format(
#            r_new.shape, v_new.shape, list_r[t+1].shape, list_v[t+1].shape))

#        list_r[t+1] = r_new.reshape(-1)
#        list_v[t+1] = v_new.reshape(-1)
#    return list_r, list_v

def Temperature(v):
    return 16*np.mean([(v_i[0]**2)+(v_i[1]**2)+(v_i[2]**2) for v_i in v])

for t in tqdm(range(num_steps)):
        print(f"HAllo {t}")
        #bytes(str(list_r[-1]), 'utf-8')
        r_new, v_new = verlet_step(list_r[t], list_v[t])
        list_r.append(r_new)
        list_v.append(v_new)

performance hint: C:\Users\corin\.ipython\cython\_cython_magic_03d7d3411128b998d63ecc60211a5e764a8dce0b.pyx:61:28: Index should be typed for more efficient access


Content of stdout:
_cython_magic_03d7d3411128b998d63ecc60211a5e764a8dce0b.c
   Creating library C:\Users\corin\.ipython\cython\Users\corin\.ipython\cython\_cython_magic_03d7d3411128b998d63ecc60211a5e764a8dce0b.cp311-win_amd64.lib and object C:\Users\corin\.ipython\cython\Users\corin\.ipython\cython\_cython_magic_03d7d3411128b998d63ecc60211a5e764a8dce0b.cp311-win_amd64.exp
Generating code
Finished generating code

  0%|          | 0/1000 [00:00<?, ?it/s]

HAllo 0
F_r
F_next


  0%|          | 1/1000 [00:22<6:11:16, 22.30s/it]

...

done
HAllo 5
F_r
F_next


  1%|          | 6/1000 [02:13<6:10:41, 22.38s/it]

done
HAllo 6
F_r


  1%|          | 6/1000 [02:19<6:24:21, 23.20s/it]


KeyboardInterrupt: 

In [ ]:
T = 2.0
sigma = 3.405 #Angstrom
epsilon = 119.8 #K
m = 48 # Be careful, m is also defined in the cython

r_0 = build_fcc()

tp = np.sqrt(48*epsilon/(m*sigma**2)) #*t

t_max = 1000
#r_total, v_total = verlet_algorithm(r_0, v_0, t_max)

In [ ]:
r_0.shape

In [ ]:
"""
r_0: Initial positions of the system
    r_0[i]: 1D-array with 3 values, the coordinates of the i'th particle
v_0: Initial velocities of the system particles
    v_0[i]: 1D-array with 3 values, the velocity values of the i'th particle
"""

In [ ]:
print(N_0)

In [ ]:
#Seeing how the initial state looks like
#ax = plt.figure().add_subplot(projection='3d')

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Extract x, y, and z coordinates
x = r_0[:,0].flatten()
y = r_0[:,1].flatten()
z = r_0[:,2].flatten()

# Plot the points
ax.scatter(x, y, z, c='r', marker='o', alpha=0.5)

# Set labels
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

# Set the title
ax.set_title('Initial state of the Argon ')

# Show the plot
#plt.show()

In [ ]:
t_max = 1000

r_total, v_total = verlet_algorithm(r_0, v_0, t_max)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.plot(r_total[:,0,0], r_total[:,0,1], r_total[:,0,2], alpha=0.5, markersize= 10, label= "particle 0")
ax.plot(r_total[:,1,0], r_total[:,1,1], r_total[:,1,2], alpha=0.5, markersize=10, label= "particle 1")

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('Trajectories ')
plt.legend()

In [ ]:
T_total = [Temperature(v_t) for v_t in v_total]
plt.plot(np.arange(t_max+1)*tp, T_total)
plt.title("Temperature over time")
plt.xlabel("$t^*$")
plt.ylabel("$T^*$")

In [ ]:
T_total

In [ ]:
flattened_array_r = r_total.reshape(-1, r_total.shape[-1])
flattened_array_v = v_total.reshape(-1, v_total.shape[-1])

# Save the flattened array to a CSV file

np.savetxt("r_total.txt", flattened_array_r, delimiter=",")
np.savetxt("v_total.txt", flattened_array_v, delimiter=",")

In [ ]:
plt.plot(np.arange(t_max+1)*tp, [F(r_total[t])[0,0] for t in range(t_max+1)])
plt.title("Force that acts on particle 0 in x-direction over time")

In [ ]:
float_value = 3.14
bytes_value = bytes(str(float_value), 'utf-8')
print(bytes_value)